<a href="https://colab.research.google.com/github/increpare/tatoeba_toki_pona_spellcheck/blob/main/tatoeba_toki_pona_spellcheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install emoji

# by jan Inkepa - https://github.com/increpare/tatoeba_toki_pona_spellcheck/blob/main/tatoeba_toki_pona_spellcheck.ipynb

import os
import urllib.request
from IPython.display import Markdown, display
from emoji import UNICODE_EMOJI

print("downloading toki pona sentences from tatoeba")
urllib.request.urlretrieve('https://downloads.tatoeba.org/exports/per_language/toki/toki_sentences_detailed.tsv.bz2', 'toki_sentences_detailed.tsv.bz2')

!rm -rf /content/toki_sentences_detailed.tsv

print("decompressing data")
!bunzip2 /content/toki_sentences_detailed.tsv.bz2
print("done")


downloading toki pona sentences from tatoeba
decompressing data
done


In [7]:
import pandas as pd
df = pd.read_csv("/content/toki_sentences_detailed.tsv",names=["k","v","user","added","modified"],delimiter="\t")
words = ["a","akesi","ala","ali","alasa","ale","anpa","ante","anu","awen","e","en","esun","ijo","ike","ilo","insa","jaki","jan","jelo","jo","kala","kalama","kama","kasi","ken","kepeken","kili","kin","kipisi","kiwen","ko","kon","kule","kulupu","kute","la","lape","laso","lawa","leko","len","lete","li","lili","linja","lipu","loje","lon","luka","lukin","lupa","ma","mama","mani","meli","mi","mije","moku","moli","monsi","monsuta","mu","mun","musi","mute","namako","nanpa","nasa","nasin","nena","ni","nimi","noka","o","oko","olin","ona","open","pakala","pali","palisa","pan","pana","pi","pilin","pimeja","pini","pipi","poka","poki","pona","pu","sama","seli","selo","seme","sewi","sijelo","sike","sin","sina","sinpin","sitelen","sona","soweli","suli","suno","supa","suwi","tan","taso","tawa","telo","tenpo","toki","tomo","tonsi","tu","unpa","uta","utala","walo","wan","waso","wawa","weka","wile"]

In [8]:
import re

def has_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count >= 1:
            return True
    return False

def validate(w):
  if w[0].isupper():
    return True#w.lower() not in words
  return w in words or w.isnumeric()

errors={}

#checks for "mi/sina" li sentences, and roughly for sentences with an "e" but without a "li"/"o"/"mi"/"sina"
def validate_sentence(s,tokens,index,user):
  if user not in errors:
    errors[user]=[]

  if (tokens[0]=="mi" or tokens[0]=="sina") and tokens[1]=="li":
    errors[user].append([s,index,"mi\sina li"])
    pass
  if "li" not in tokens and "e" in tokens and "o" not in tokens:
    if "mi" not in tokens and "sina" not in tokens:      
      errors[user].append([s,index,"'e' without 'li'/'mi'/'sina'/'o'"])
      pass
  if bool(re.search(r"\b(la,?|\.) (mi|sina) li\b",s)):
    errors[user].append([s,index,"mi\sina li"])
    pass
  if bool(re.search(r"(wu|wo|ji|ti)",s)):
    errors[user].append([s,index,"forbidden syllables wu/wo/ji/ti"])
    pass
  lastchar = s[len(s)-1]
  if lastchar.isalnum():
    errors[user].append([s,index,"Sentence doesn't end with punctuation"])
    pass
  # if has_emoji(s):
  #  errors[user].append([s,index,"Contains emoji"])

freq={}
links={}
id_table={}

from ipywidgets import IntProgress
from IPython.display import display
f = IntProgress(min=0, max=len(df),description="searching...") # instantiate the bar

display(f) # display the bar


i=0
for index, row in df.iterrows():
  import re
  id_table[index]=row
  user = row['user']
  sentence=row['v']
  tokens = re.findall(r'\w+',sentence)
  validate_sentence(sentence,tokens,index,user)
  for w in tokens:
    if not validate(w):
      if ('"'+w+'"') not in sentence:
        if w not in freq:
          freq[w]=0
          links[w]=[]
        freq[w]=freq[w]+1
        links[w].append(index)
        errors[user].append([sentence,index,"likely misspelled word: "+w])
  i=i+1
  if (i%1000)==0:
          f.value=i

import datetime

output="# Tatoeba Toki Pona Error Report\n\n"+str(datetime.datetime.now())+"\n\n"

def printm(s):
  global output
  output=output+"\n"+s;

for u in errors:
  errorlist = errors[u]
  if len(errorlist)==0:
    continue
  printm("\n")
  printm("***")
  printm(" \n")
  printm("### Likely Error report for user "+u)
  printm("\n")
  for error in errorlist:
    printm(""+error[0]+"  ")
    printm("- "+str(error[2])+"  ")
    printm("- http://tatoeba.org/eng/sentences/show/"+str(error[1])+"  ")
    printm("")

display(Markdown(output))

IntProgress(value=0, description='searching...', max=42235)

# Tatoeba Toki Pona Error Report

2020-12-02 19:51:12.736983




***
 

### Likely Error report for user kroko


„ona li open!“ – „ala. sina li open!“  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/2969963  

sina wile ala wile wan e nanpa ni  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/3005286  

mi pona e tomo pi pata mi.  
- likely misspelled word: pata  
- http://tatoeba.org/eng/sentences/show/3005615  

pona! lawa insa sina li pini tawa sina pi nanpa tu. sina li kama moli li pona.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/3080251  

pata mi li jo e soweli.  
- likely misspelled word: pata  
- http://tatoeba.org/eng/sentences/show/3938433  

mi kama jo e kili Apo teka tu lon esun.  
- likely misspelled word: teka  
- http://tatoeba.org/eng/sentences/show/4600833  



***
 

### Likely Error report for user bojnin


jan lili mute li musi lon ma kasi pi ma tomo  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/642739  

tenpo suno kama la seli li pona la mi tawa tan insa  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/667636  

pilin pi kasi kule Lila li lon ma kasi pi poka tomo mute  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/740208  



***
 

### Likely Error report for user negativeclock


jan mute li olin e jan ni pi kalama pona sin ala  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/1419652  



***
 

### Likely Error report for user Tepan


jan Wope Teniwo li kama lon sitelen tawa ni lon tenpo lili.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/9159880  



***
 

### Likely Error report for user Wadimiy


pata suli li lukin e sina.  
- likely misspelled word: pata  
- http://tatoeba.org/eng/sentences/show/2907834  



***
 

### Likely Error report for user rkqb


tempo suno kama la jan Kise li ken ala pali.  
- likely misspelled word: tempo  
- http://tatoeba.org/eng/sentences/show/1703362  



***
 

### Likely Error report for user carlosalberto


jan lawa nanpa wan pi kule pimeja pi ma Mewika li jan Palakawopama.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/4543481  

jan Awowi li tawa musi pona mute.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/4545010  

mi sona e ni: sina kama sona mute  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/4640786  

jan Sunewoki Iketa li pona tawa ona.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/4674880  

jan Siwokene li jan sona pi ma Elena li jo ala e tomo. tomo ona li poki sike pi tenpo suli.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/4712141  

nasin pona ii pona mute ala.  
- likely misspelled word: ii  
- http://tatoeba.org/eng/sentences/show/9135950  



***
 

### Likely Error report for user soweli_Elepanto


jan Losi li musi uta ala  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/1725107  



***
 

### Likely Error report for user komap


mo jo e palisa luka mute.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/9140426  

mo jo e palisa luka mute.  
- likely misspelled word: mo  
- http://tatoeba.org/eng/sentences/show/9140426  



***
 

### Likely Error report for user temlat


telo Waloti li telo nasa ala. ona li telo kasi seli.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/2622442  



***
 

### Likely Error report for user klagefall


mi en meli mi li wan  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/3002378  

kute tan kama sona. kute tan ala toki  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/3002384  



***
 

### Likely Error report for user Reonaato


jan pona kute e jan sewi.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6634557  



***
 

### Likely Error report for user pinaxe


mi pilin e ni: tenpo kama li pona ala anu li pona lili  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6174026  

ijo in li ijo sina ala sina?  
- likely misspelled word: in  
- http://tatoeba.org/eng/sentences/show/6177177  



***
 

### Likely Error report for user list


soweli pi mu "uuu-aaa-a-a-a" li jo e sona wawa.  
- likely misspelled word: uuu  
- http://tatoeba.org/eng/sentences/show/6058474  

soweli pi mu "uuu-aaa-a-a-a" li jo e sona wawa.  
- likely misspelled word: aaa  
- http://tatoeba.org/eng/sentences/show/6058474  

jan Mawi li moku e kili loje Tomato  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6372734  

mi wile sona e ni: tenpo pini la tan seme la jan Ton li kama jo lon esun e ni  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6376397  

jan Ton li jo e sijelo ike. ona li toki sama waso pimeja Kowo.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6376658  

tenpo ale ona la ona pana e pona tawa jan.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6376694  

tenpo muno li jo e tenpo suno li mute seme?  
- likely misspelled word: muno  
- http://tatoeba.org/eng/sentences/show/6381347  

jan ni li weka ike e sitlen lape pi mi mute la jan ni li lawa e moli tawa mi mute.  
- likely misspelled word: sitlen  
- http://tatoeba.org/eng/sentences/show/6391233  

jan 🤠 li wile kama sona. taso sina sama soweli 🐃 la sina li kama sona pi wawa mute.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6391330  

pip Luso seme li tawa lon insa Epato sina?  
- likely misspelled word: pip  
- http://tatoeba.org/eng/sentences/show/6392362  

jan li toki tan akesi ike suli 👹 la jan Kati li kama.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6397359  

sina pakala ala e jan ante la sina ken pali e wile sina  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6401642  

ma tomo Situtika li kama pakala la seme li lon?  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6406004  

jan Mawi li jan pali pi sitlen tawa li jo e ken.  
- likely misspelled word: sitlen  
- http://tatoeba.org/eng/sentences/show/6452080  

pona la moku e moku pona e telo pona la ike la toki nasa.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6457104  

tomo lipu 🏭📰 li kama moli la nasin Temokati li kama moli.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6466795  

tenpo ala la jan Tane li kama sona ala e mama sina  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6467190  

tenpo wan pini la  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6469732  

kili pomo li lon ni. mi lukin e ona.  
- likely misspelled word: pomo  
- http://tatoeba.org/eng/sentences/show/6480018  

olin li wile ala kama jo tan jan ante. olin li wille pana e ali tawa jan ante.  
- likely misspelled word: wille  
- http://tatoeba.org/eng/sentences/show/6480036  

mi mute li pali e anu tawa ni: kama weka lon lape la ona li ken pali e sitlen lape ona tawa lon.  
- likely misspelled word: sitlen  
- http://tatoeba.org/eng/sentences/show/6482929  

tenpo sinpin la sina kepeken e ilo sitlen tawa la o pali e pali pi kama sona.  
- likely misspelled word: sitlen  
- http://tatoeba.org/eng/sentences/show/6534496  

sitelen pi nimi ma li selo tan palisa pi kasi suli la jan li ken lili lukin e sitlen ni.  
- likely misspelled word: sitlen  
- http://tatoeba.org/eng/sentences/show/6544005  

mi jan Anitile.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6578583  

akesi ike suli 👹 li kama jo e wan pi linja lawa sina la tenpo ale la sina li jo pi akesi ike suli.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6587590  

lon pi mel ni li suwi.  
- likely misspelled word: mel  
- http://tatoeba.org/eng/sentences/show/6590967  

jan Putin li wile e nasin seme pi kulupu jan li lawa tawa lon?  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6595743  

mi wile ala pakala e jan ali  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6596965  

mute seme pi len noka anpa la sina li jo?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6602498  

tenpo monsi pi tenpo sike pi tu wan la meli ni li tawa weka tan jan olin la tan ni la jan oli li kepeken ike e meli ni.  
- likely misspelled word: oli  
- http://tatoeba.org/eng/sentences/show/6617669  

nimi mi li jan Lutiwi.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6626836  

open ala. tan ni la pona e tomo.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6667614  

wile la walo moku Salito tawa e seli moku.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6669160  

lon seme la kama e pakala?  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6682102  

mi kama jo ala e sono sin tan jan Tone.  
- likely misspelled word: sono  
- http://tatoeba.org/eng/sentences/show/6688317  

pon sina o lon seme li poki telo jaki?  
- likely misspelled word: pon  
- http://tatoeba.org/eng/sentences/show/6700528  

lon seme la ken lukin e sitelen tawa?  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6700617  

lon seme la sina li tomo tawa?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6700663  

mi lukin lon sona mi la sina li mije sama mi.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6702656  

sijelo pi jan Ton li suli mute la sielo pi jan Mawi li suli lili.  
- likely misspelled word: sielo  
- http://tatoeba.org/eng/sentences/show/6843418  

jan Ton li pana e pono tawa mama meli ona lon tomo moku.  
- likely misspelled word: pono  
- http://tatoeba.org/eng/sentences/show/6847920  

suno li sono walo mute kin la mi mute li wile e lukin tawa poka.  
- likely misspelled word: sono  
- http://tatoeba.org/eng/sentences/show/6851919  

tenop suno ni la mi wile ala musi e musi Teniso.  
- likely misspelled word: tenop  
- http://tatoeba.org/eng/sentences/show/6855920  

ni li pona lukin lon sitelten. ni li pona lili lon lon.  
- likely misspelled word: sitelten  
- http://tatoeba.org/eng/sentences/show/6862159  

mi wile ala e ni: ona mijel li kama tawa ma pi akesi Satan. taso mi ken ala weka e ike tan ona.  
- likely misspelled word: mijel  
- http://tatoeba.org/eng/sentences/show/6886624  

pona li ni: tomo sin en ja pona pi tenpo suli.  
- likely misspelled word: ja  
- http://tatoeba.org/eng/sentences/show/6891252  

sina kepeken e nena lili ni la lupo tomo li kama open.  
- likely misspelled word: lupo  
- http://tatoeba.org/eng/sentences/show/6892286  

ona li tomo lon tomo pi mani mute. poka lili lon ma laso kulupu Senta Pawo.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6893979  

jan li kekepen kasi tawa supa sitelen.  
- likely misspelled word: kekepen  
- http://tatoeba.org/eng/sentences/show/6895331  

tenpo pini la mi lape pona ala  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6897354  

ilo tawa mi li tenpo suli mute la mi li tenpo suli lili.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6899857  

tenpo mute la jan suli mute Atila li lukin sama e ni: ona li jo e sike ma lon sijelo Sulito.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6899877  

tawa mi la tenpo mun pi nanpa luka tu tu li mute ale lon tenpo sike  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6900215  

jan li toki e ni: nimi a. la ona li wile toki kin e ni: nimi p. ona li pana lon tenpo wan la ona li wile pana kin lon tenpo tu.  
- likely misspelled word: p  
- http://tatoeba.org/eng/sentences/show/6931594  

tenpo ala la toki e ni: ala!  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6931709  

sina ken ala kepeken e toki lon ala tawa mi. mi lukin e ni: tenpo pini la jan pali utala li lon ni kin. taso ona li kama jo ike ala e ijo. sina li jan pi kama jo ike. sina wile pana e mani.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6934826  

mi sona e ni: sina jo ala e ijo ni. mi kama pana e mani mute tawa sina. taso mi wile sona e ni: ken la sina li jan pi pilin akesi la mi wile ala pana e mani tawa ala.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6934830  

toki olin taso la kepeken e sike pi palisa luka.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6943316  

telo sijeo loje pi tu wan li tawa anpa lon supa pi ko lete.  
- likely misspelled word: sijeo  
- http://tatoeba.org/eng/sentences/show/6955304  

waso li kama tawa lukin e jan lili. waso Siko e waso Kowo e waso Kolonpo.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6955423  

pona nasa la lupa telo li jo ala e telo. jan pi tenpo sike lili li tawa anpa pakala lon kasi lili Muko taso. taso ona li ken tawa ala tan lupo.  
- likely misspelled word: lupo  
- http://tatoeba.org/eng/sentences/show/6963589  

suno lili taso tawa lukin e lipu.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6981367  

telo loje siejelo li lon len sina anu seme?  
- likely misspelled word: siejelo  
- http://tatoeba.org/eng/sentences/show/6981492  

jan seme la ken pini e ni?  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6994307  

wawa lili sina seme la sina li jo?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6996679  

tan seme la sina li jan pi pona ale tawa pali ni?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7001171  

tenpo mute la sina li ike sijelo anu seme?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7001334  

tawa jan ante la mi jan lawa. taso mi ala kin lon kupupu.  
- likely misspelled word: kupupu  
- http://tatoeba.org/eng/sentences/show/7009772  

ale ll jaki lon sike mi.  
- likely misspelled word: ll  
- http://tatoeba.org/eng/sentences/show/7020066  

sina lukin ala e pona lili pi tenpo lili lo o pini.  
- likely misspelled word: lo  
- http://tatoeba.org/eng/sentences/show/7030039  

tawa jan Ton la jan Mali li ike lukin. taso jan Mali li ken kama lon pona lukln.  
- likely misspelled word: lukln  
- http://tatoeba.org/eng/sentences/show/7039112  

jan Toni en jan Ton li utala sama e jan Tawi en jan Koli  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7054245  

temo ala la jan Mali li moku e kiwen insa en selo pi kili Pomo.  
- likely misspelled word: temo  
- http://tatoeba.org/eng/sentences/show/7059362  

sina jo ala jo e lete pi ike siejelo?  
- likely misspelled word: siejelo  
- http://tatoeba.org/eng/sentences/show/7109069  

jan Ton li open e kalama musi uta ona  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7115418  

pilin sina la sina li meli pi pona ale lukin lon sike ma.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7128361  

tenpo suo open ale la jan Ton li awen lon ma kasi.  
- likely misspelled word: suo  
- http://tatoeba.org/eng/sentences/show/7223669  

meli li suli  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7247231  

tenpo suno ni la jan Mali li seli e moku sike Patikuko.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7255562  

telo naso Wino loje ni li pona ale tawa mi.  
- likely misspelled word: naso  
- http://tatoeba.org/eng/sentences/show/7372885  

sina wile ala wile moku e kili pomo? - wile ala. mi moku e kili tu lon tenpo pini.  
- likely misspelled word: pomo  
- http://tatoeba.org/eng/sentences/show/7461691  

tenpo 🎄 la mi li tawa tomo pi jan lili mi.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7477348  

tawa moku la jan Ton li moku e telo naso pi poki tu taso.  
- likely misspelled word: naso  
- http://tatoeba.org/eng/sentences/show/7477703  

soweli li anpa lo supa monsi la sina ken ala ken lukin e ona?  
- likely misspelled word: lo  
- http://tatoeba.org/eng/sentences/show/7487904  

tomo tawa ni li jaki la oni li wile e pona.  
- likely misspelled word: oni  
- http://tatoeba.org/eng/sentences/show/7666648  

wile mir la ken la sina sama li ken toki e ale tawa mi.  
- likely misspelled word: mir  
- http://tatoeba.org/eng/sentences/show/7680675  

jan pi poka seme la sina li lon?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7703411  

ona li kama sona e toki Latino lon tomo sona.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7713814  

pilin mi la sina kepeken e len ni la sina li pona lukin.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7758284  

nasina ala tawa ni: jan li ken weka tan lon ni.  
- likely misspelled word: nasina  
- http://tatoeba.org/eng/sentences/show/7768376  

sina moku ala molu e telo nasa Wino? - ala. mi kepeken ilo tawa la mi moku e telo kili Pomo.  
- likely misspelled word: molu  
- http://tatoeba.org/eng/sentences/show/7786322  

tenpo pini la sina li jan pali pi tomo moku anu seme?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7816890  

mi wile e nasin ni: tan tomo awen pi jan kama tawa tomo awen pi tomo tawa lijna.  
- likely misspelled word: lijna  
- http://tatoeba.org/eng/sentences/show/7817169  

jan Ton li wile ala kepeken ilo pi sitele tawa.  
- likely misspelled word: sitele  
- http://tatoeba.org/eng/sentences/show/7818990  

nasin seme la nasin Politiko li pali lon ma Kanse?  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7820227  

sina lukin sama e ni: sina pilin nasa la tan seme la sina li ni?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7821481  

mi will pona e jan Ton tan ni: ona li ken tawa tomo sona.  
- likely misspelled word: will  
- http://tatoeba.org/eng/sentences/show/7829288  

tenpo seme la sina kama lon suno? - tenpo mun pi nanpa tu tu la tenpo suno pi luka wan  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7848143  

o tenpo pimjea open pona tawa meli en mije mi. o kama pona lon sitelen tawa pi mi mute.  
- likely misspelled word: pimjea  
- http://tatoeba.org/eng/sentences/show/7848213  

tenpo ilo pi napa tu wan la jan Ton li kama lon tomo awen pi tomo tawa linja.  
- likely misspelled word: napa  
- http://tatoeba.org/eng/sentences/show/7874828  

nimi ni li kama tan toki Latin.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7874896  

seme li ma Palatiso tawa sina?  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7883926  

m tawa tomo mi. tenpo lili la mi kama sin.  
- likely misspelled word: m  
- http://tatoeba.org/eng/sentences/show/7887619  

mi lon poka pi nasin Temokati.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7887868  

nasin Temokati li pona tawa mi.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7887869  

tan seme la sina kepeken e supa mons mi?  
- likely misspelled word: mons  
- http://tatoeba.org/eng/sentences/show/7894628  

ilo ni pi sitele tawa li pakala.  
- likely misspelled word: sitele  
- http://tatoeba.org/eng/sentences/show/7919275  

sama ni la sina wile kama jo e mi la mi li kiwen tawa sina.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7928258  

mi pilin sin sama ni: mi jan pi tenpo sike lili  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7960997  

sina ala lon ni la tan ni: mi mute li pilin ike  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7965476  

sitelten sona ona li suli ala.  
- likely misspelled word: sitelten  
- http://tatoeba.org/eng/sentences/show/8178509  

o tawa. sina sama o luki pali.  
- likely misspelled word: luki  
- http://tatoeba.org/eng/sentences/show/8178970  

o tenpo pimjea pona. o sitelen lape tawa sina.  
- likely misspelled word: pimjea  
- http://tatoeba.org/eng/sentences/show/8182663  

tenpo suno pini ni la mije sama pi mama mi li moli tan ike sielo Kanselo.  
- likely misspelled word: sielo  
- http://tatoeba.org/eng/sentences/show/8184588  

unu. tu, tu wan. tu tu. luka. luka wan. luka tu. luka tu wan. luka tu tu. luka luka.  
- likely misspelled word: unu  
- http://tatoeba.org/eng/sentences/show/8198974  

sina lon sina la sina li pona tawa mi.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/8209740  

ja Lisa li wile ala lon kulupu.  
- likely misspelled word: ja  
- http://tatoeba.org/eng/sentences/show/8220556  

tenpo pi ike sijo ona la jan pona li kama tawa ona lon tenpo mute.  
- likely misspelled word: sijo  
- http://tatoeba.org/eng/sentences/show/8220603  

sina sama e ni la sina li pona tawa mi.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/8223348  

telo siejlo loje li telo pi pona mute.  
- likely misspelled word: siejlo  
- http://tatoeba.org/eng/sentences/show/8225254  

a! ona li kalam musi ike! jaki! ona li kama musi pi nasin lawa!  
- likely misspelled word: kalam  
- http://tatoeba.org/eng/sentences/show/8225289  

teno suno sin ale la jan Ton li pali lukin e ijo ante.  
- likely misspelled word: teno  
- http://tatoeba.org/eng/sentences/show/8306566  

tenpo ala la jan Ton li toki eni.  
- likely misspelled word: eni  
- http://tatoeba.org/eng/sentences/show/8333292  

tenpo mute la mije sama mi li awen lape ala lon tenpo pimjea.  
- likely misspelled word: pimjea  
- http://tatoeba.org/eng/sentences/show/8612710  

ona li jo ala e wawa pi sitlene sona.  
- likely misspelled word: sitlene  
- http://tatoeba.org/eng/sentences/show/8612757  

jan tawa pi ma ante li kute e toki pi jan lawa  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/8621360  

sina lukin ala lukin e sitele lawa pi ona mije?  
- likely misspelled word: sitele  
- http://tatoeba.org/eng/sentences/show/8622798  

ona li seli e suwi pan Mupin  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/8622807  

tenpo suno napa pi luka wan li tenpo suno musi ona.  
- likely misspelled word: napa  
- http://tatoeba.org/eng/sentences/show/8666221  

teno ni la jan seme li lon sewi?  
- likely misspelled word: teno  
- http://tatoeba.org/eng/sentences/show/8666320  

sina sona ala sona wawa e ni: tomo tawa lijna ni li pona?  
- likely misspelled word: lijna  
- http://tatoeba.org/eng/sentences/show/8670991  

kili mute lon pokl.  
- likely misspelled word: pokl  
- http://tatoeba.org/eng/sentences/show/8677007  

mi lon seme lon nasin ni. o lawa e lukin mi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/8688368  

mama mama meli mi o lawa e lukin mi tan ni: nasin seme la mi li pona e supa lape mi?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/8690719  

mi lawa e ilo tawa sike 🚲 la tepo ale la mi kepeken selo lawa ⛑.  
- likely misspelled word: tepo  
- http://tatoeba.org/eng/sentences/show/8891402  

tenpo suli seme la tomo esun li open lon tenp suno ni?  
- likely misspelled word: tenp  
- http://tatoeba.org/eng/sentences/show/8900890  

mije ni li utata e ona. ona li wile moli e ona.  
- likely misspelled word: utata  
- http://tatoeba.org/eng/sentences/show/8908424  

jan Mali li kama jo e supo monsi ni: ona li poka lili lon lupa tomo.  
- likely misspelled word: supo  
- http://tatoeba.org/eng/sentences/show/8908570  

nunpa 0️⃣ li pona tawa jan lawa pi tomo tawa telo. ona li pilin e ni: nanpa 0️⃣ li sama e ilo sike lon telo suli.  
- likely misspelled word: nunpa  
- http://tatoeba.org/eng/sentences/show/8987116  

tenpo pmeja pini ni la mi moku e pan en sin Kuli 🍛.  
- likely misspelled word: pmeja  
- http://tatoeba.org/eng/sentences/show/8987363  

sitelen sona li kama ala la ni li sama. jan li wile sitelten la jan li ken.  
- likely misspelled word: sitelten  
- http://tatoeba.org/eng/sentences/show/8995854  

tenpo pimjea la kon tawa 🌬 li tawa wawa.  
- likely misspelled word: pimjea  
- http://tatoeba.org/eng/sentences/show/9048938  

len noko 🧦 pi jan Ton li jo e lupa 🕳.  
- likely misspelled word: noko  
- http://tatoeba.org/eng/sentences/show/9051020  

janTon li alasa e kala la ni li pona tawa ona.  
- likely misspelled word: janTon  
- http://tatoeba.org/eng/sentences/show/9067218  

temp ale la jan sewi en jan anpa li kama lon tenpo sama.  
- likely misspelled word: temp  
- http://tatoeba.org/eng/sentences/show/9117066  

jan li jo e mani pi mute ili la jan li jo e jan lili pi mute pona.  
- likely misspelled word: ili  
- http://tatoeba.org/eng/sentences/show/9120460  



***
 

### Likely Error report for user Rodrigopredja


tenpo pimeja pini la mi sitelen lape e sina uta tawa uta mi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/4975267  



***
 

### Likely Error report for user Zyurs


jan Mike ken ala pali e ijo sama ni.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6403400  

tenpo ali la jan ken pali e pali pi jan ante.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6485448  

ilo lukin ona li pakala  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6589582  

jan pi kalama musi li lon ala  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6589637  



***
 

### Likely Error report for user jan_Ne


o tawa  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864665  

mi tawa  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864671  

mi toki kepeken toki Epanja  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864683  

mi toki kepeken toki Inli  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864685  

ma Mewika li lon poka ma Kanata  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864692  

ike ale ni li tan mi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7758436  

jan Tan li pana e suwi pimeja tawa jan Linta  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7778509  

pipi li lon tenpo lili  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7785521  

o kama lon poka mi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7893457  

pona tawa sina  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/8129773  



***
 

### Likely Error report for user Goatstein


len noka ni lli len kiwen.  
- likely misspelled word: lli  
- http://tatoeba.org/eng/sentences/show/7775311  

tenpo pini la, jan Ton ken ala lukin e jan.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/7792134  

tenpo pini la, mi kama sona e ni: ma li tawa wawa lon ma tomo Awaji.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7792671  

ona li weka e kulupu mama ona li tawa lon ma Tajiti.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7797653  

tenpo pimjea wan la, mi mute li ken ala ken awen lon tomo sina?  
- likely misspelled word: pimjea  
- http://tatoeba.org/eng/sentences/show/7797765  

nimi "anticonstitutionnellement" li jo e nimi lili luka luka luka luka luka li nimi suli mute pi toki Kanse.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7798710  

mama pi tenpo pini li sewi tawa kulupi ni.  
- likely misspelled word: kulupi  
- http://tatoeba.org/eng/sentences/show/7799863  

tenpo pini la, mi moku mute. mi pana e mani lili  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7805078  

tenpo suno al la, seli li suli mute.  
- likely misspelled word: al  
- http://tatoeba.org/eng/sentences/show/7806808  

teno suno pini ni la, lon tenpo ali la, ona li moku e telo nasa.  
- likely misspelled word: teno  
- http://tatoeba.org/eng/sentences/show/7808296  

tenpo seme la sina li weka e lon ni?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7810236  

tenpo suno kama ni la, seli e mute seme?  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/7815138  

jan Ton li weka e ino tenpo ona.  
- likely misspelled word: ino  
- http://tatoeba.org/eng/sentences/show/7818266  

jan Peti o, ni li mije lili sina anu seme?  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7827545  

jan Ton il wile moli e sina tan seme?  
- likely misspelled word: il  
- http://tatoeba.org/eng/sentences/show/7833611  

poki li jo e kili liji loje mute.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7834553  

poki li jo e kili liji loje mute.  
- likely misspelled word: liji  
- http://tatoeba.org/eng/sentences/show/7834553  

jan Peti li telo e kasi kule.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7836199  

awen lon kulupu linja li ike mute awa mi.  
- likely misspelled word: awa  
- http://tatoeba.org/eng/sentences/show/7840617  

sike suno ni la, mi mute li jo e ko lete mute. sike suno pini la, mi mute li joe ko lete lili.  
- likely misspelled word: joe  
- http://tatoeba.org/eng/sentences/show/7842362  

tenpo pini lila la, mi toki e ona tawa jan ton.  
- likely misspelled word: lila  
- http://tatoeba.org/eng/sentences/show/7866735  

tenpo pini lila la, mi toki e ona tawa jan ton.  
- likely misspelled word: ton  
- http://tatoeba.org/eng/sentences/show/7866735  

jan Pati li kalama musi li tawa musi.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7868365  

jan Kanti li toki lon.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7868502  

ma Awaji li ma pona kin.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7868504  

tenpo ijo la, sina li seli ala seli e ilo nasa pi kiwen wawa?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7869710  

mi wile tawa ma Awaiji.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7956237  

jan Sami li lon ma tomo Pakati.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7977227  

tenpo kin la, o lukin tawa e moku sama lon telo suli ali. kala ali li moku e kali ali, li utala ali tan tenpo pi ali open.  
- likely misspelled word: kali  
- http://tatoeba.org/eng/sentences/show/8001227  

sine wile e ijo seme?  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/8069210  

sine wile e ijo seme?  
- likely misspelled word: sine  
- http://tatoeba.org/eng/sentences/show/8069210  

jan mute li sona e sitelen jan Tatinka pi jan Lotin.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/8084875  

nimi mi li nimi Osimantija. mi jan lawa pi jan lawa. jan wawa o, lukin e pali mi, o pilin anpa a!  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/8086711  

mi pilin e ni: musi Kantiku en musi sama li ike tawa tenpo.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/8104755  

toki Alape non e ike mute.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/8115071  

toki Alape non e ike mute.  
- likely misspelled word: non  
- http://tatoeba.org/eng/sentences/show/8115071  



***
 

### Likely Error report for user Ilia2


jan pi ma Losi li nimi "ъ" sama "твёрдый знак". ona li nimi "ь" sama "мягкий знак".  
- likely misspelled word: твёрдый  
- http://tatoeba.org/eng/sentences/show/7887101  

jan pi ma Losi li nimi "ъ" sama "твёрдый знак". ona li nimi "ь" sama "мягкий знак".  
- likely misspelled word: знак  
- http://tatoeba.org/eng/sentences/show/7887101  

jan pi ma Losi li nimi "ъ" sama "твёрдый знак". ona li nimi "ь" sama "мягкий знак".  
- likely misspelled word: мягкий  
- http://tatoeba.org/eng/sentences/show/7887101  

jan pi ma Losi li nimi "ъ" sama "твёрдый знак". ona li nimi "ь" sama "мягкий знак".  
- likely misspelled word: знак  
- http://tatoeba.org/eng/sentences/show/7887101  



***
 

### Likely Error report for user epiceric9


ona li weka e kulupu mama ona li tawa lon ma Tawiti.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/8116919  



***
 

### Likely Error report for user shou


pona  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/8719275  



***
 

### Likely Error report for user Aleks


Tom ĉiutage lavas siajn dentojn.  
- likely misspelled word: ĉiutage  
- http://tatoeba.org/eng/sentences/show/9083237  

Tom ĉiutage lavas siajn dentojn.  
- likely misspelled word: lavas  
- http://tatoeba.org/eng/sentences/show/9083237  

Tom ĉiutage lavas siajn dentojn.  
- likely misspelled word: siajn  
- http://tatoeba.org/eng/sentences/show/9083237  

Tom ĉiutage lavas siajn dentojn.  
- likely misspelled word: dentojn  
- http://tatoeba.org/eng/sentences/show/9083237  

tenpo pini la, mi lukin e jan lawa meli  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/9085360  



***
 

### Likely Error report for user Yl


mi tan ma Sonko  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/9131306  

jan Ale li tonsi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/9131889  

lipu ni li jo la, jan tu li ken kama insa  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/9132036  



***
 

### Likely Error report for user eltacodemex


sina toki e toki katala, anu seme?  
- likely misspelled word: katala  
- http://tatoeba.org/eng/sentences/show/9178220  



***
 

### Likely Error report for user Sobsz


sina ken ala powe e mi.  
- likely misspelled word: powe  
- http://tatoeba.org/eng/sentences/show/9331471  



***
 

### Likely Error report for user inkepa


waso li awen lon sike, la wasli lili sin li kama tan ona.  
- likely misspelled word: wasli  
- http://tatoeba.org/eng/sentences/show/9354130  

mi moki ala e ni.  
- likely misspelled word: moki  
- http://tatoeba.org/eng/sentences/show/9381359  

jan ton li jan pi anpa mute.  
- likely misspelled word: ton  
- http://tatoeba.org/eng/sentences/show/9392947  

jan Pati li anpa, li sona.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/9392949  


In [12]:

with open('/content/output.MD','w') as writefile: 
 writefile.write(output)
print("file written")


file written


In [11]:
with open('drive/MyDrive/tatoeba/test.txt', 'w') as writeFile:
  writeFile.write("test")